In [1]:
import numpy as np
from vidaug import augmentors as va
import matplotlib.pyplot as plt
import cv2
import os
import random
from pathlib import Path

In [9]:
#Set up file location
RWF_dataset_dir = "/home/amrmarey/Work/ECE740_Project/RWF_2000"  #location of original dataset
RWF_sets = ['train']
RWF_output_classes = ['Fight', 'NonFight']
RWF_Aug_dir = "/home/amrmarey/Work/ECE740_Project/RWF_2000_Flipped"

In [10]:
####This is for just flipping
for RWF_set in RWF_sets: #Augment both train and val
    for RWF_output_class in RWF_output_classes: #Augment each class Fight and NonFight
        folder_path = RWF_dataset_dir +'/' + RWF_set + '/' + RWF_output_class
        print(folder_path)
        # Create a Path object for the folder
        folder = Path(folder_path)
        
        # Get a list of all files in the folder
        file_list = [file for file in folder.iterdir() if file.is_file()]
        out_dir = RWF_Aug_dir + '/' + RWF_set + '/' + RWF_output_class
        for file in file_list:
            #Location of original file in computer
            video_path = RWF_dataset_dir +'/' + RWF_set+ '/'+ RWF_output_class+ '/' + str(file).split("/")[-1]
            
            #Location of wher you would like the output file to be
            out_path = out_dir + '/' + str(file).split("/")[-1][:-4] + '_flipped.avi'
            # Load the video file
            cap = cv2.VideoCapture(video_path)
            # Define the output video writer
            # Meta.
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_size = (width, height)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            # Video writer.
            out = cv2.VideoWriter(out_path, fourcc, fps, frame_size)

            # collect all frames of the video
            frames = []
            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret == True:
                    frames.append(frame)
                else:
                    break
            cap.release()
            #Data augmentations to consider
            seq = va.Sequential([va.HorizontalFlip()])
            #augment the frames
            video_aug = seq(frames)

            # output the video
            for frame in video_aug:
                out.write(frame)
            out.release()

            
print('Done!!!!!!!!')

/home/amrmarey/Work/ECE740_Project/RWF_2000/train/Fight
/home/amrmarey/Work/ECE740_Project/RWF_2000/train/NonFight
Done!!!!!!!!


In [11]:
#The rest of the script is responsible for noise injection
RWF_dataset_dir = "/home/amrmarey/Work/ECE740_Project/RWF_2000" #location of original dataset
RWF_sets = ['train']
RWF_output_classes = ['Fight', 'NonFight']
RWF_Aug_dir_v2 = "/home/amrmarey/Work/ECE740_Project/RWF_2000_Aug" #location of augmented dataset

In [12]:
#Noise injection for nonflipped videos
M = 1
while M < 3: #Run through the dataset twice
    for RWF_set in RWF_sets:
        for RWF_output_class in RWF_output_classes:
            folder_path = RWF_dataset_dir +'/' + RWF_set + '/' + RWF_output_class
            # Create a Path object for the folder
            folder = Path(folder_path)

            # Get a list of all files in the folder
            file_list = [file for file in folder.iterdir() if file.is_file()]
            out_dir = RWF_Aug_dir_v2 + '/' + RWF_set + '/' + RWF_output_class
            for file in file_list:
                #Location of original file in computer
                video_path = RWF_dataset_dir +'/' + RWF_set+ '/'+ RWF_output_class+ '/' + str(file).split("/")[-1]
                
                #Location of wher you would like the output file to be
                out_path = out_dir + '/' + str(file).split("/")[-1][:-4] + '_'+ str(M)+'.avi'
                # Load the video file
                cap = cv2.VideoCapture(video_path)
                # Define the output video writer
                # Meta.
                fps = int(cap.get(cv2.CAP_PROP_FPS))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                frame_size = (width, height)
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                # Video writer.
                out = cv2.VideoWriter(out_path, fourcc, fps, frame_size)

                # collect all frames of the video
                frames = []
                while(cap.isOpened()):
                    ret, frame = cap.read()
                    if ret == True:
                        frames.append(frame)
                    else:
                        break
                cap.release()
                # Apply the video augmentation pipeline to each frame of the video
                
                #Different Augmentations considered. Each video is randomly augmented with different augmentation and each augmentation has a random intensity
                salt_pepper_ratio = random.randint(5, 90)
                salt_and_pepper = va.Sequential(transforms = [(va.Salt(ratio = salt_pepper_ratio)),(va.Pepper(ratio = salt_pepper_ratio))], random_order = True)
                seq = va.SomeOf(transforms = [va.GaussianBlur(sigma = random.uniform(1, 8)), salt_and_pepper, va.Multiply(value=random.uniform(0.85, 1.15))], N = random.randint(1, 3), random_order=True)

                #augment the frames
                video_aug = seq(frames)

                # output the video
                for frame in video_aug:
                    out.write(frame)
                out.release()
            
    M = M+1
    print(M)

            
print('Done!!!!!!!!')

2
3
Done!!!!!!!!


In [13]:
RWF_dataset_dir = "/home/amrmarey/Work/ECE740_Project/RWF_2000_Flipped"
RWF_sets = ['train']
RWF_output_classes = ['Fight', 'NonFight']
RWF_Aug_dir_v2 = "/home/amrmarey/Work/ECE740_Project/RWF_2000_Aug"

In [14]:
#Noise injection for flipped videos (everything is same as above except folder locations change)
M = 1
while M < 3: #Run through the dataset twice
    for RWF_set in RWF_sets:
        for RWF_output_class in RWF_output_classes:
            folder_path = RWF_dataset_dir +'/' + RWF_set + '/' + RWF_output_class
            # Create a Path object for the folder
            folder = Path(folder_path)

            # Get a list of all files in the folder
            file_list = [file for file in folder.iterdir() if file.is_file()]
            out_dir = RWF_Aug_dir_v2 + '/' + RWF_set + '/' + RWF_output_class
            for file in file_list:
                #Location of original file in computer
                video_path = RWF_dataset_dir +'/' + RWF_set+ '/'+ RWF_output_class+ '/' + str(file).split("/")[-1]
                
                #Location of wher you would like the output file to be
                out_path = out_dir + '/' + str(file).split("/")[-1][:-4] + '_'+ str(M)+'.avi'
                # Load the video file
                cap = cv2.VideoCapture(video_path)
                # Define the output video writer
                # Meta.
                fps = int(cap.get(cv2.CAP_PROP_FPS))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                frame_size = (width, height)
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                # Video writer.
                out = cv2.VideoWriter(out_path, fourcc, fps, frame_size)

                # collect all frames of the video
                frames = []
                while(cap.isOpened()):
                    ret, frame = cap.read()
                    if ret == True:
                        frames.append(frame)
                    else:
                        break
                cap.release()
                # Apply the video augmentation pipeline to each frame of the video
                
                #Different Augmentations considered. Each video is randomly augmented with different augmentation and each augmentation has a random intensity
                salt_pepper_ratio = random.randint(5, 90)
                salt_and_pepper = va.Sequential(transforms = [(va.Salt(ratio = salt_pepper_ratio)),(va.Pepper(ratio = salt_pepper_ratio))], random_order = True)
                seq = va.SomeOf(transforms = [va.GaussianBlur(sigma = random.uniform(1, 8)), salt_and_pepper, va.Multiply(value=random.uniform(0.85, 1.15))], N = random.randint(1, 3), random_order=True)

                #augment the frames
                video_aug = seq(frames)

                # output the video
                for frame in video_aug:
                    out.write(frame)
                out.release()
            
    M = M+1
    print(M)

            
print('Done!!!!!!!!')

2
3
Done!!!!!!!!
